# Data Augmentation

- Artificiellt skapar mer bilder, när vi har lite data
- Slumpmässigt roterar till en viss grad (radianer)
- Slumpämssigt translatera
- Slumpmässigt flippa horisontellt, vertikalt (spegla)
- shear / skjuvning
- ...

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.datasets.mnist import load_data

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

In [ ]:
def plot_samples(data, nrows=2, ncols=5, figsize=(12, 4)):
    
    fig, axes = plt.subplots(nrows, ncols, figsize = figsize)

    for i, ax in enumerate(axes.flatten()):
        ax.imshow(data[i,:,:], cmap="gray") #0-9 and all rows and columns
        ax.axis("off")

    fig.subplots_adjust(wspace=0, hspace=.1, bottom=0)

In [ ]:
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255
X_test.min(), X_test.max()

### Train | val | test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_val, y_train_val, y_val = train_test_split(
    X_train, y_train, test_size=1/6, random_state = 42)

X_train_val = X_train_val[:,:,:,None]
X_train = X_train[:,:,:,None]
X_val = X_val[:,:,:,None]
X_test = X_test[:,:,:,None]

X_train_val.shape, X_val.shape, y_train_val.shape, y_val.shape

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(
                        rotation_range=10,
                        shear_range=.2,
                        zoom_range=.1,
                        horizontal_flip=False, 
                        height_shift_range=.2, 
                        width_shift_range=.2
                        )

# Don't augment validation and test data
test_image_generator = ImageDataGenerator()

train_val_generator = train_image_generator.flow(X_train_val, y_train_val, batch_size=32)

val_generator = test_image_generator.flow(X_val, y_val, batch_size=32)

train_val_generator, val_generator

In [ ]:
print(len(train_val_generator.next())) #next is an iterator object, .next brings us to the next batch

sample_batch = train_val_generator.next() #32 samples with 28*28 pixels in our first batch
print(sample_batch[0].shape)

plot_samples(sample_batch[0])
sample_batch[1]

### CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

def CNN_model(learning_rate=.001, drop_rate=.5, kernels=[32, 32]):
    adam = Adam(learning_rate = learning_rate)

    model = Sequential(name="CNN_model")

    # The convolutional layers
    for number_kernel in kernels:
        conv_layer = Conv2D(number_kernel, 
                            kernel_size=(3, 3), 
                            activation="relu", 
                            kernel_initializer="he_normal", 
                            input_shape=X_train.shape[1:])
        
        model.add(conv_layer)
        model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    
    # MLP layers
    model.add(Flatten())
    model.add(Dropout(drop_rate))
    model.add(Dense(256, activation="relu", kernel_initializer="he_normal"))
    model.add(Dense(10, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy", optimizer=adam, metrics=["acc"])

    return model

model = CNN_model(drop_rate=.5)
model.summary()    

### Train on augmented data

In [ ]:
steps_per_epoch = int(len(X_train_val)/32)
validation_steps = int(len(X_val)/32)

steps_per_epoch, validation_steps

#We make sure that it will train on all of the data, since we use mini batch

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor = "val_acc", mode = "max", patience = 5, restore_best_weights = True)

model.fit(train_val_generator, 
         steps_per_epoch = steps_per_epoch, 
         epochs = 100, 
         callbacks = [early_stopper], 
         validation_data=val_generator, 
         validation_steps = validation_steps)

In [ ]:
def plot_metrics(metrics):
    _, ax = plt.subplots(1,2, figsize = (12,4))
    metrics[["loss", "val_loss"]].plot(ax = ax[0], title = "Loss", grid = True)
    metrics[["acc", "val_acc"]].plot(ax = ax[1], title = "Accuracy", grid = True)

metrics = pd.DataFrame(model.history.history)
plot_metrics(metrics)

### Train on all training data

In [ ]:
train_generator = train_image_generator.flow(X_train, y_train, batch_size=32)

In [ ]:
model = CNN_model()
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs = 15)

### Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot()
